In [ ]:
import os
import numpy as np
import moviepy.editor as mpy
import subprocess
from pose_features import load_eface, compute_features, extract_features
from vis_tools import segment_exps, plot_features, plot_scores, animate_features, FEATURES_LIST, EXPS_LIST
from IPython.display import Video

FPS = 30

In [ ]:
root_dir = '/pasteur/u/mottaghi/datasets/facial_paralysis'
case_id = 'CompleteFlaccid1'
features2vis = [4, 1, 6]

In [ ]:
eface_file = os.path.join(root_dir, case_id, f'eface_{case_id}.json')
eface = load_eface(eface_file)
pose_file = os.path.join(root_dir, case_id, f'pose_{case_id}.npy')
pose = np.load(pose_file, allow_pickle=True).item()
features = extract_features(pose)
length = len(features[0]) / FPS
segments = segment_exps(eface, length)
segments

In [ ]:
feat_file = os.path.join(root_dir, case_id, f'feats_{case_id}.jpg')
plot_features(features, eface, out_path=feat_file, show=True)

In [ ]:
from sklearn.preprocessing import Normalizer
import moviepy.editor as mpy
from matplotlib import pyplot as plt
import cv2
import numpy as np

cap = cv2.VideoCapture(os.path.join(root_dir, case_id, f'video_{case_id}.mp4'))
frames_list = []
while True:
    success, frame = cap.read()
    if not success:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames_list.append(frame)

for j, exp in enumerate(EXPS_LIST):
    start, end = tuple(eface['Facial Expressions'][exp])
    exp_features = np.zeros_like(features)
    exp_features[:, start*FPS:end*FPS] = features[:, start*FPS:end*FPS]
    exp_trans= Normalizer().fit(exp_features)
    exp_norm_features = exp_trans.transform(np.abs(exp_features))
    exp_max_loc = exp_norm_features.sum(axis=0).argmax()
    exp_frame = frames_list[exp_max_loc]
    plt.imshow(exp_frame)
    plt.axis('off')
    plt.show()
    plt.imsave(os.path.join(root_dir, case_id, f'{case_id}-{j}_keyframe.jpg'), exp_frame)

In [ ]:
score_file = os.path.join(root_dir, case_id, f'{case_id}_scores.jpg')
plot_scores(eface, out_path=score_file, show=True)

In [ ]:
ani_file = os.path.join(root_dir, case_id, f'feats_{case_id}.mp4')
animate_features(100*features[features2vis[0]], 100*features[features2vis[1]], 100*features[features2vis[2]], segments, out_path=ani_file, labels=[FEATURES_LIST[features2vis[0]], FEATURES_LIST[features2vis[1]], FEATURES_LIST[features2vis[2]]])

In [ ]:
vis_clip = mpy.VideoFileClip(os.path.join(root_dir, case_id, f'vis_{case_id}.mp4'))
feat_clip = mpy.VideoFileClip(ani_file)
score_clip = mpy.VideoFileClip(score_file)
mpy.clips_array([[vis_clip, feat_clip]]).write_videofile(os.path.join(root_dir, case_id, f'demo_{case_id}.mp4'))